In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('tabela.csv', sep=';')
df.shape

(10757, 31)

In [3]:
df = df.dropna(subset=['nome_alimento'])
# df

In [4]:
df.isnull().sum()

id_alimento                 0
id_usuario               1970
nome_alimento               0
estado_alimento             0
alimento_verificado      1970
grupo_alimentar             0
marca_alimento           1970
alimento_ativo           1970
dtt_criacao_alimento     1970
id_tabela_nutricional    1970
unidade_medida           1970
porcao_padrao            1970
kcal                        0
qtde_proteina               0
qtde_gordura                0
qtde_carboidrato            0
qtde_alcool              1947
qtde_fibra                  0
qtde_saturada               0
qtde_monosaturada           0
qtde_polisaturada           0
qtde_trans                  0
qtde_acucar                 0
qtde_calcio                 0
qtde_ferro                  0
qtde_sodio                  0
qtde_potassio               0
qtde_vitamina_a             0
qtde_vitamina_d             0
qtde_vitamina_e             0
qtde_vitamina_c             0
dtype: int64

In [5]:
df['alimento_verificado'] = 'TRUE'
df['alimento_ativo'] = 'TRUE'
df['dtt_criacao_alimento'] = '2024-08-29 00:00:00'
df['unidade_medida'] = 'GRAMA'
df['porcao_padrao'] = 100
df.head()

,id_alimento,id_usuario,nome_alimento,estado_alimento,alimento_verificado,grupo_alimentar,marca_alimento,alimento_ativo,dtt_criacao_alimento,id_tabela_nutricional,...,qtde_trans,qtde_acucar,qtde_calcio,qtde_ferro,qtde_sodio,qtde_potassio,qtde_vitamina_a,qtde_vitamina_d,qtde_vitamina_e,qtde_vitamina_c
0,6300101.0,NaN,"Arroz (polido, parboilizado, agulha, agulhinha...",Não se aplica,TRUE,VEGANO,NaN,TRUE,2024-08-29 00:00:00,NaN,...,"0,01",-,"3,51","0,08","1,19","14,53",-,-,"0,08",-
1,6300201.0,NaN,Arroz integral,Não se aplica,TRUE,VEGANO,NaN,TRUE,2024-08-29 00:00:00,NaN,...,"0,01",-,"5,15","0,26","1,23","74,42",-,-,"0,08",-
2,6300701.0,NaN,Milho (em grão),Cru(a),TRUE,VEGANO,NaN,TRUE,2024-08-29 00:00:00,NaN,...,"0,03","3,16","3,15","0,45","244,96","212,05","13,17",-,"0,57","6,20"
3,6300701.0,NaN,Milho (em grão),Cozido(a),TRUE,VEGANO,NaN,TRUE,2024-08-29 00:00:00,NaN,...,"0,03","3,16","3,15","0,45","244,96","212,05","13,17",-,"0,57","6,20"
4,6300701.0,NaN,Milho (em grão),Grelhado(a)/brasa/churrasco,TRUE,VEGANO,NaN,TRUE,2024-08-29 00:00:00,NaN,...,"0,03","3,16","3,15","0,45","244,96","212,05","13,17",-,"0,57","6,20"


In [6]:
for col in df.columns:
    df[col] = df[col].replace('-', '')

In [7]:
df['kcal'] = df['kcal'].replace('', pd.NA)
df['kcal'] = df['kcal'].fillna(0)

df['qtde_carboidrato'] = df['qtde_carboidrato'].replace('', pd.NA)
df['qtde_carboidrato'] = df['qtde_carboidrato'].fillna(0)

df['qtde_proteina'] = df['qtde_proteina'].replace('', pd.NA)
df['qtde_proteina'] = df['qtde_proteina'].fillna(0)

df['qtde_gordura'] = df['qtde_gordura'].replace('', pd.NA)
df['qtde_gordura'] = df['qtde_gordura'].fillna(0)

df['qtde_gordura'] = df['qtde_gordura'].replace('', pd.NA)
df['qtde_alcool'] = df['qtde_alcool'].fillna(0)

(df[['kcal', 'qtde_proteina', 'qtde_gordura', 'qtde_carboidrato', 'qtde_alcool']] == '').sum()

kcal                0
qtde_proteina       0
qtde_gordura        0
qtde_carboidrato    0
qtde_alcool         0
dtype: int64

In [8]:
df['nome_alimento'] = df['nome_alimento'].str.replace(r'\s+', ' ', regex=True).str.strip()
df

,id_alimento,id_usuario,nome_alimento,estado_alimento,alimento_verificado,grupo_alimentar,marca_alimento,alimento_ativo,dtt_criacao_alimento,id_tabela_nutricional,...,qtde_trans,qtde_acucar,qtde_calcio,qtde_ferro,qtde_sodio,qtde_potassio,qtde_vitamina_a,qtde_vitamina_d,qtde_vitamina_e,qtde_vitamina_c
0,6300101.0,NaN,"Arroz (polido, parboilizado, agulha, agulhinha...",Não se aplica,TRUE,VEGANO,NaN,TRUE,2024-08-29 00:00:00,NaN,...,"0,01",,"3,51","0,08","1,19","14,53",,,"0,08",
1,6300201.0,NaN,Arroz integral,Não se aplica,TRUE,VEGANO,NaN,TRUE,2024-08-29 00:00:00,NaN,...,"0,01",,"5,15","0,26","1,23","74,42",,,"0,08",
2,6300701.0,NaN,Milho (em grão),Cru(a),TRUE,VEGANO,NaN,TRUE,2024-08-29 00:00:00,NaN,...,"0,03","3,16","3,15","0,45","244,96","212,05","13,17",,"0,57","6,20"
3,6300701.0,NaN,Milho (em grão),Cozido(a),TRUE,VEGANO,NaN,TRUE,2024-08-29 00:00:00,NaN,...,"0,03","3,16","3,15","0,45","244,96","212,05","13,17",,"0,57","6,20"
4,6300701.0,NaN,Milho (em grão),Grelhado(a)/brasa/churrasco,TRUE,VEGANO,NaN,TRUE,2024-08-29 00:00:00,NaN,...,"0,03","3,16","3,15","0,45","244,96","212,05","13,17",,"0,57","6,20"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1965,8579003.0,NaN,Maria izabel,Não se aplica,TRUE,ONIVORO,NaN,TRUE,2024-08-29 00:00:00,NaN,...,,,"13,00","1,00","1622,00","87,00",,,,
1966,8579004.0,NaN,Arroz de leite,Não se aplica,TRUE,ONIVORO,NaN,TRUE,2024-08-29 00:00:00,NaN,...,"0,05","1,47","38,74","0,96","391,85","66,44","7,80","0,27","0,30","0,01"
1967,8579005.0,NaN,Arroz com mandioca,Não se aplica,TRUE,ONIVORO,NaN,TRUE,2024-08-29 00:00:00,NaN,...,"0,00",,"11,25","0,09","1,09","57,26",,,,"5,55"
1968,8579006.0,NaN,Arroz com ovo,Não se aplica,TRUE,ONIVORO,NaN,TRUE,2024-08-29 00:00:00,NaN,...,"0,02","0,66","28,93","0,69","67,99","75,74","92,03","0,70","0,81",


In [9]:
df['nome_alimento'] = df['nome_alimento'].str.replace('não especificada', '', regex=False).str.strip()
df['nome_alimento'] = df['nome_alimento'].str.replace('não especificado', '', regex=False).str.strip()
# df

In [10]:
# print(df.estado_alimento.unique())
df.loc[df['estado_alimento'].str.contains('Não se aplica', case=False, na=False), 'estado_alimento'] = 'PADRAO'
df.loc[df['estado_alimento'].str.contains('Cru', case=False, na=False), 'estado_alimento'] = 'CRU'
df.loc[df['estado_alimento'].str.contains('Cozido', case=False, na=False), 'estado_alimento'] = 'COZIDO'
df.loc[df['estado_alimento'].str.contains('Grelhado', case=False, na=False), 'estado_alimento'] = 'GRELHADO'
df.loc[df['estado_alimento'].str.contains('Assado', case=False, na=False), 'estado_alimento'] = 'ASSADO'
df.loc[df['estado_alimento'].str.contains('Refogado', case=False, na=False), 'estado_alimento'] = 'REFOGADO'
df.loc[df['estado_alimento'].str.contains('Frito', case=False, na=False), 'estado_alimento'] = 'FRITO'

print(df.estado_alimento.unique())

['PADRAO' 'CRU' 'COZIDO' 'GRELHADO' 'ASSADO' 'REFOGADO' 'Ensopado'
 'Mingau' 'Sopa' 'FRITO' 'Molho vermelho' 'Molho branco' 'Ao alho e óleo'
 'Com manteiga/óleo' 'Empanado(a)/à milanesa' 'Ao vinagrete']


In [11]:
valores_validos = ['PADRAO', 'CRU', 'COZIDO', 'GRELHADO', 'ASSADO', 'REFOGADO', 'FRITO']
df = df[df['estado_alimento'].isin(valores_validos)]
print(df.estado_alimento.unique())

['PADRAO' 'CRU' 'COZIDO' 'GRELHADO' 'ASSADO' 'REFOGADO' 'FRITO']


In [12]:
print(df.shape)
df = df.drop_duplicates(subset=['nome_alimento', 'estado_alimento'], keep='first')
print(df.shape)

(1673, 31)
(1672, 31)


In [13]:
print(df.shape)
df = df[~((df['estado_alimento'] == 'PADRAO') & (df['nome_alimento'].duplicated(keep=False)))]
print(df.shape)

(1672, 31)
(1609, 31)


In [14]:
df [ df['nome_alimento'].str.contains(',', case=False, na=False) ]

,id_alimento,id_usuario,nome_alimento,estado_alimento,alimento_verificado,grupo_alimentar,marca_alimento,alimento_ativo,dtt_criacao_alimento,id_tabela_nutricional,...,qtde_trans,qtde_acucar,qtde_calcio,qtde_ferro,qtde_sodio,qtde_potassio,qtde_vitamina_a,qtde_vitamina_d,qtde_vitamina_e,qtde_vitamina_c
0,6300101.0,NaN,"Arroz (polido, parboilizado, agulha, agulhinha...",PADRAO,TRUE,VEGANO,NaN,TRUE,2024-08-29 00:00:00,NaN,...,"0,01",,"3,51","0,08","1,19","14,53",,,"0,08",
27,6303102.0,NaN,"Feijão (preto, mulatinho, roxo, rosinha, etc.)",PADRAO,TRUE,VEGANO,NaN,TRUE,2024-08-29 00:00:00,NaN,...,"0,01","0,30","55,20","2,22","5,20","336,60",,,"0,69",
438,6801101.0,NaN,"Banana (ouro, prata, d´água, da terra, etc.)",CRU,TRUE,VEGANO,NaN,TRUE,2024-08-29 00:00:00,NaN,...,,"12,23","5,00","0,26","1,00","358,00","3,21",,"0,13","8,70"
439,6801101.0,NaN,"Banana (ouro, prata, d´água, da terra, etc.)",COZIDO,TRUE,VEGANO,NaN,TRUE,2024-08-29 00:00:00,NaN,...,,"12,23","5,00","0,26","1,00","358,00","3,21",,"0,13","8,70"
440,6801101.0,NaN,"Banana (ouro, prata, d´água, da terra, etc.)",ASSADO,TRUE,VEGANO,NaN,TRUE,2024-08-29 00:00:00,NaN,...,,"12,23","5,00","0,26","1,00","358,00","3,21",,"0,13","8,70"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1946,8508401.0,NaN,"Salada ou verdura crua, exceto de fruta",PADRAO,TRUE,ONIVORO,NaN,TRUE,2024-08-29 00:00:00,NaN,...,,"2,62","10,00","0,27","5,00","237,00","41,63",,"0,54","12,70"
1959,8571110.0,NaN,"Carne com batata, inhame, batata-baroa ou aipim",PADRAO,TRUE,ONIVORO,NaN,TRUE,2024-08-29 00:00:00,NaN,...,"0,36","0,43","8,00","1,56","36,00","332,50","0,08","0,35","0,32","3,70"
1960,8571111.0,NaN,"Carne com legumes (exceto batata, inhame e aipim)",PADRAO,TRUE,ONIVORO,NaN,TRUE,2024-08-29 00:00:00,NaN,...,"0,35","1,65","11,50","1,69","34,00","283,50","124,79","0,35","0,65","2,35"
1961,8571112.0,NaN,"Frango com batata, inhame batata-baroa ou aipim",PADRAO,TRUE,ONIVORO,NaN,TRUE,2024-08-29 00:00:00,NaN,...,"0,06","0,43","11,50","0,69","41,00","287,50","4,58","0,10","0,39","3,70"


In [15]:
def convert_and_truncate(x):
    try:
        return round(float(x.replace(',', '.')), 1)
    except (ValueError, AttributeError):
        return x

df = df.applymap(convert_and_truncate)

/var/folders/40/ncqy45yx5y59gm95g5v_kx480000gn/T/ipykernel_5872/373288614.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(convert_and_truncate)


In [16]:
df['id_alimento'] = range(1, len(df) + 1)
df['id_tabela_nutricional'] = df['id_alimento']

In [17]:
# colunas_float = valores = [ "id_tabela_nutricional", "kcal", "qtde_proteina", "qtde_carboidrato", "qtde_gordura", "qtde_alcool", "qtde_acucar", "qtde_fibra", "qtde_saturada", "qtde_monosaturada",  "qtde_polisaturada", "qtde_trans", "qtde_sodio", "qtde_calcio", "qtde_ferro", "qtde_potassio", "qtde_vitamina_a", "qtde_vitamina_c", "qtde_vitamina_d", "qtde_vitamina_e"
# ]
# for col in colunas_float:
#     df[col] = pd.to_numeric(df[col], errors='coerce')

# colunas_com_strings_vazias = df.applymap(lambda x: isinstance(x, str) and x == '').any()

# # Exibir o resultado
# print(colunas_com_strings_vazias)

df [df.qtde_gordura.isnull()]['qtde_gordura']

Series([], Name: qtde_gordura, dtype: float64)

In [18]:
df.to_csv('TabelaNutrIA-Tratada.csv', sep=',', index=False)

In [19]:
df_alimentos = df[['id_alimento', 'id_usuario', 'nome_alimento', 'estado_alimento', 'alimento_verificado', 'grupo_alimentar', 'marca_alimento', 'dtt_criacao_alimento', 'alimento_ativo']]
df_alimentos.to_csv('TabelaNutrIA-Alimentos.csv', sep=',', index=False)

In [20]:
colunas_diferenca = df.columns.difference(df_alimentos.columns)

df_tabela_nutricional = df[colunas_diferenca]
df_tabela_nutricional['id_alimento'] = df['id_alimento']
df_tabela_nutricional.to_csv('TabelaNutrIA-TabelaNutricional.csv', sep=',', index=False)

/var/folders/40/ncqy45yx5y59gm95g5v_kx480000gn/T/ipykernel_5872/3474774222.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tabela_nutricional['id_alimento'] = df['id_alimento']
